### Naive Bayes

In [180]:
X_train_gau = X_train[["temperature"]]

In [181]:
X_train_ber = X_train.drop(columns=["Age", "date_diff", "temperature", "total_visit_num"], axis=1)

In [182]:
X_train_mul = X_train[["Age", "date_diff", "total_visit_num"]]

In [183]:
from imblearn.under_sampling import *

In [184]:
X_samp_gau, y_samp_gau = EditedNearestNeighbours(kind_sel="all", n_neighbors=5, random_state=0).fit_sample(X_train_gau, y_train)

In [185]:
X_samp_ber, y_samp_ber = EditedNearestNeighbours(kind_sel="all", n_neighbors=5, random_state=0).fit_sample(X_train_ber, y_train)

In [186]:
X_samp_mul, y_samp_mul = EditedNearestNeighbours(kind_sel="all", n_neighbors=5, random_state=0).fit_sample(X_train_mul, y_train)

In [187]:
X_test_gau = X_test[["temperature"]]

In [188]:
X_test_ber = X_test.drop(columns=["Age", "date_diff", "temperature", "total_visit_num"], axis=1)

In [189]:
X_test_mul = X_test[["Age", "date_diff", "total_visit_num"]]

### GaussianNB

In [190]:
from sklearn.naive_bayes import GaussianNB
model_norm = GaussianNB().fit(X_samp_gau, y_samp_gau)

In [191]:
y_pred_gau = model_norm.predict(X_test_gau)

In [192]:
y_pred_gau_prob = model_norm.predict_proba(X_test_gau)

In [193]:
gau_prob = y_pred_gau_prob[:,1]

In [194]:
confusion_matrix(y_test, y_pred_gau, labels=["Yes", "No"])

array([[  723,  5971],
       [ 2389, 24073]])

In [195]:
print(classification_report(y_test, y_pred_gau))

              precision    recall  f1-score   support

          No       0.80      0.91      0.85     26462
         Yes       0.23      0.11      0.15      6694

   micro avg       0.75      0.75      0.75     33156
   macro avg       0.52      0.51      0.50     33156
weighted avg       0.69      0.75      0.71     33156



### BernoulliNB

In [196]:
from sklearn.naive_bayes import BernoulliNB
model_bern = BernoulliNB().fit(X_samp_ber, y_samp_ber)

In [197]:
y_pred_ber = model_bern.predict(X_test_ber)

In [198]:
y_pred_ber_prob = model_bern.predict_proba(X_test_ber)

In [199]:
ber_prob = y_pred_ber_prob[:,1]

In [200]:
confusion_matrix(y_test, y_pred_ber, labels=["Yes", "No"])

array([[ 4820,  1874],
       [11205, 15257]])

In [201]:
print(classification_report(y_test, y_pred_ber))

              precision    recall  f1-score   support

          No       0.89      0.58      0.70     26462
         Yes       0.30      0.72      0.42      6694

   micro avg       0.61      0.61      0.61     33156
   macro avg       0.60      0.65      0.56     33156
weighted avg       0.77      0.61      0.64     33156



### MultinomialNB

In [202]:
from sklearn.naive_bayes import MultinomialNB
model_mult = MultinomialNB().fit(X_samp_mul, y_samp_mul)

In [203]:
y_pred_mul = model_mult.predict(X_test_mul)

In [204]:
y_pred_mul_prob = model_mult.predict_proba(X_test_mul)

In [205]:
mul_prob = y_pred_mul_prob[:,1]

In [206]:
confusion_matrix(y_test, y_pred_mul, labels=["Yes", "No"])

array([[ 3925,  2769],
       [ 8306, 18156]])

In [207]:
print(classification_report(y_test, y_pred_mul))

              precision    recall  f1-score   support

          No       0.87      0.69      0.77     26462
         Yes       0.32      0.59      0.41      6694

   micro avg       0.67      0.67      0.67     33156
   macro avg       0.59      0.64      0.59     33156
weighted avg       0.76      0.67      0.70     33156



### GaussianNB + BernoulliNB + MultinomialNB Probability

https://stackoverflow.com/questions/33477736/how-to-combine-the-outputs-of-multiple-naive-bayes-classifier

In [243]:
# pos = pos_prior_ * (y_prob1[:,1]/pos_prior_) * (y_prob2[:,1]/pos_prior_) * (y_prob3[:,1]/pos_prior_) * (y_prob4[:,1]/pos_prior_)
# pos = y_prob1[:,1] * y_prob2[:,1] * y_prob3[:,1] * y_prob4[:,1] / pos_prior_**3
neg = (y_pred_gau_prob[:,0] * y_pred_ber_prob[:,0] * y_pred_mul_prob[:,0]) / (pos_prior ** 2)
pos = (gau_prob * ber_prob * mul_prob) / (pos_prior ** 2)

In [250]:
prob = pos / (neg + pos)

In [322]:
y_pred_total = [ "No" if p < 0.04 else "Yes" for p in prob ]

In [323]:
accuracy_score(y_test, y_pred_total)

0.6383460007238508

In [324]:
confusion_matrix(y_test, y_pred_total, labels=["Yes", "No"])

array([[ 4469,  2225],
       [ 9766, 16696]])

In [325]:
print(classification_report(y_test, y_pred_total))

              precision    recall  f1-score   support

          No       0.88      0.63      0.74     26462
         Yes       0.31      0.67      0.43      6694

   micro avg       0.64      0.64      0.64     33156
   macro avg       0.60      0.65      0.58     33156
weighted avg       0.77      0.64      0.67     33156

